<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/DeepSeek_Airline_Operation_Control_Agent_(Fully_Automated_Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import time

# Import userdata from google.colab for secure API key management
from google.colab import userdata

def control_airline_operations(scenario: str) -> str:
    """
    Analyzes an airline operational scenario using the DeepSeek-Reasoner model,
    simulating a multi-agent system with 5 specialized agents for operation control.

    Args:
        scenario (str): The airline operational scenario to be analyzed.

    Returns:
        str: A comprehensive, step-by-step operational analysis from 5 perspectives.
    """
    # Attempt to retrieve the API key from Colab's userdata secrets
    try:
        deepseek_api_key = userdata.get("DEEPSEEK_API_KEY")
    except Exception as e:
        return f"Error: Could not retrieve DeepSeek API Key from Colab secrets. Please ensure it's set in the 'Secrets' panel. Details: {e}"

    if not deepseek_api_key:
        return "Error: DeepSeek API Key is not set in Colab secrets. Please set 'DEEPSEEK_API_KEY' in the 'Secrets' panel."

    if not scenario.strip():
        return "Error: Please provide an airline operational scenario to analyze."

    # Construct the prompt for the deepseek-reasoner model.
    # This prompt defines the roles of the 5 sub-agents and the desired output structure.
    prompt = f"""You are a highly analytical AI airline operations agent. Your task is to process an airline operational scenario and provide a comprehensive, step-by-step analysis, acting as five different specialized sub-agents. Each sub-agent will contribute their specific expertise to the overall operational assessment and recommended actions.

Here is the scenario:
{scenario}

Your analysis should follow these sequential steps, with each step representing the output of a distinct sub-agent:

1.  **Flight Status Monitor**:
    * **Role**: Gathers and synthesizes all relevant real-time data related to flights mentioned (e.g., flight numbers, origins, destinations, scheduled/actual times, weather conditions, NOTAMs - Notices to Airmen, aircraft status).
    * **Output**: Provide a concise summary of the current operational status and any immediate environmental factors.

2.  **Resource Allocator**:
    * **Role**: Based on the current flight status and any disruptions, evaluates the allocation of key resources, including available aircraft, crew assignments, and gate availability. Proposes optimal re-allocations if necessary.
    * **Output**: Detail resource utilization and suggest any re-allocation strategies.

3.  **Disruption Management Agent**:
    * **Role**: Identifies any operational disruptions (e.g., delays, cancellations, diversions, mechanical issues, airspace closures) and assesses their direct impact on the affected flights and the broader schedule. Proposes immediate mitigation actions.
    * **Output**: Describe the disruption, its impact, and initial mitigation steps.

4.  **Passenger Impact Assessor**:
    * **Role**: Quantifies and evaluates the impact of any operational disruptions on passengers (e.g., number of affected passengers, missed connections, overnight stays). Suggests passenger communication strategies and re-accommodation options.
    * **Output**: Summarize passenger impact and recommend communication/re-accommodation plans.

5.  **Safety & Compliance Auditor**:
    * **Role**: Reviews the scenario and proposed actions for adherence to aviation safety regulations (e.g., crew duty limits, aircraft maintenance requirements), company operational policies, and legal compliance. Flags any potential violations or risks.
    * **Output**: Provide a compliance check and highlight any safety considerations.

Ensure your output is structured with clear headings for each agent's analysis.
"""

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {deepseek_api_key}"
    }

    payload = {
        "model": "deepseek-reasoner",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "stream": False # We are not using streaming for this example
    }

    api_url = "https://api.deepseek.com/chat/completions" # DeepSeek API endpoint for chat completions

    # Implement exponential backoff for API calls to handle rate limits.
    max_retries = 5
    delay = 1  # Start with 1 second delay
    for i in range(max_retries):
        try:
            response = requests.post(api_url, headers=headers, json=payload, timeout=60) # Set a timeout
            response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)

            result = response.json()

            # Check for valid response structure from DeepSeek API
            if result and result.get("choices") and len(result["choices"]) > 0 and \
               result["choices"][0].get("message") and result["choices"][0]["message"].get("content"):
                return result["choices"][0]["message"]["content"]
            else:
                return f"Error: Unexpected DeepSeek API response structure: {json.dumps(result, indent=2)}"

        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                print(f"Rate limit exceeded (HTTP 429). Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2 # Exponential backoff
            else:
                return f"HTTP error occurred: {e}"
        except requests.exceptions.ConnectionError as e:
            return f"Connection error occurred: {e}"
        except requests.exceptions.Timeout:
            return "The request timed out. Please try again."
        except requests.exceptions.RequestException as e:
            return f"An unexpected request error occurred: {e}"

    return "Failed to get analysis after multiple retries due to rate limiting or other issues."

if __name__ == "__main__":
    print("DeepSeek Airline Operation Control Agent (Fully Automated Python)")
    print("----------------------------------------------------------------")
    print("This agent will automatically analyze a predefined airline operational scenario and provide structured insights.")
    print("Please ensure your DeepSeek API Key is set as a secret named 'DEEPSEEK_API_KEY' in Google Colab's 'Secrets' panel.")

    # Predefined airline operational scenario for full automation
    automated_airline_scenario = """
    Flight AA123 (scheduled from NYC to LAX, ETD 08:00 AM, ETA 11:00 AM) is experiencing a mechanical issue requiring a 2-hour delay for maintenance. The aircraft for this flight is a Boeing 737.
    Crew for AA123: Captain Smith (duty expires 03:00 PM), First Officer Jones (duty expires 04:00 PM), 4 flight attendants.
    Expected weather at LAX is clear. NYC has light rain.
    Another flight, AA456 (LAX to MIA, scheduled ETD 12:00 PM), was supposed to use the same aircraft from AA123.
    There are 150 passengers on AA123, 80 of whom have connecting flights from LAX within 3 hours of original ETA.
    """

    print(f"\nAnalyzing scenario automatically:\n'{automated_airline_scenario}'")
    analysis = control_airline_operations(automated_airline_scenario)
    print("\n--- Operational Analysis ---")
    print(analysis)
    print("----------------------------")

    print("\nDeepSeek Airline Operation Control Agent has completed its automated analysis.")

DeepSeek Airline Operation Control Agent (Fully Automated Python)
----------------------------------------------------------------
This agent will automatically analyze a predefined airline operational scenario and provide structured insights.
Please ensure your DeepSeek API Key is set as a secret named 'DEEPSEEK_API_KEY' in Google Colab's 'Secrets' panel.

Analyzing scenario automatically:
'
    Flight AA123 (scheduled from NYC to LAX, ETD 08:00 AM, ETA 11:00 AM) is experiencing a mechanical issue requiring a 2-hour delay for maintenance. The aircraft for this flight is a Boeing 737.
    Crew for AA123: Captain Smith (duty expires 03:00 PM), First Officer Jones (duty expires 04:00 PM), 4 flight attendants.
    Expected weather at LAX is clear. NYC has light rain.
    Another flight, AA456 (LAX to MIA, scheduled ETD 12:00 PM), was supposed to use the same aircraft from AA123.
    There are 150 passengers on AA123, 80 of whom have connecting flights from LAX within 3 hours of original E